## Домашнее задание

Основаная задача - **построить хорошую тематическую модель с интерпретируемыми топиками с помощью LDA в gensim и NMF в sklearn**.


1) сделайте нормализацию (если pymorphy2 работает долго используйте mystem или попробуйте установить быструю версию - `pip install pymorphy2[fast]`, можно использовать какой-то другой токенизатор); 

2) добавьте нграммы (в тетрадке есть закомменченая ячейка с Phrases,  можно также попробовать другие способы построить нграммы); 

3) сделайте хороший словарь (отфильтруйте слишком частотные и редкие слова, попробуйте удалить стоп-слова); 

4) постройте несколько LDA моделей (переберите количество тем, можете поменять eta, alpha, passes), если получаются плохие темы, поработайте дополнительно над предобработкой и словарем; 

5) для самой хорошей модели в отдельной ячейке напечатайте 3 хороших (на ваш вкус) темы;

6) между словарем и обучением модели добавьте tfidf (`gensim.models.TfidfModel(corpus, id2word=dictionary); corpus = tfidf[corpus]`);

7) повторите пункт 4 на преобразованном корпусе;

8) в отдельной ячейке опишите как изменилась модель (приведите несколько тем, которые стали лучше или хуже, или которых раньше вообще не было; можно привести значения перплексии и когерентности для обеих моделей)

9) проделайте такие же действия для NMF (образец в конце тетрадки), для построения словаря воспользуйтесь возможностями Count или Tfidf Vectorizer (попробуйте другие значение max_features, min_df, max_df, сделайте нграмы через ngram_range, если хватает памяти), попробуйте такие же количества тем

10) в отдельной ячейки напечатайте таблицу с темами лучшей NMF модели, сравните их с теми, что получились в LDA.

Сохраните тетрадку с экспериментами и положите её на гитхаб, ссылку на неё укажите в форме.

**Оцениваться будут главным образом пункты 5, 8 и 10. (2, 3, 2 баллов соответственно). Чтобы заработать остальные 3 балла, нужно хотя бы немного изменить мой код на промежуточных этапах (добавить что-то, указать другие параметры и т.д). **

In [1]:
import gensim
import json
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
import pyLDAvis.gensim
import string
from collections import Counter
import warnings
warnings.filterwarnings("ignore")

morph = MorphAnalyzer()

## Данные

In [2]:
stops = set(stopwords.words('russian')) | {'gt',} | set(stopwords.words('english'))
def remove_tags(text):
    return re.sub(r'<[^>]+>', '', text)

def normalize(words):
    norm_words = [morph.parse(word)[0].normal_form for word in words if len(set(word)) > 1]
    return norm_words

def opt_normalize(texts, top=None):
    uniq = Counter()
    for text in texts:
        uniq.update(text)
    
    norm_uniq = {word:morph.parse(word)[0].normal_form for word, _ in uniq.most_common(top)}
    
    norm_texts = []
    for text in texts:
        
        norm_words = [norm_uniq.get(word) for word in text]
        norm_words = [word for word in norm_words if word and word not in stops]
        norm_texts.append(norm_words)
        
    return norm_texts

def tokenize(text):
    words = [word.strip(string.punctuation) for word in text.split()]
    words = [word for word in words if word]
    
    return words

Возьмем 4 тыс статьи с Хабра. Это мало для хорошей тематической модели, но иначе у нас просто ничего не обучится за семинар.

В текстах есть тэги. Потрем их. Ещё токенизируем самым простым способом и нормализуем Pymorphy.

In [3]:
%%time
texts = open('habr_texts.txt').read().splitlines()
texts = [tokenize(remove_tags(text.lower())) for text in texts]

CPU times: user 3.7 s, sys: 608 ms, total: 4.3 s
Wall time: 4.36 s


In [4]:
%%time
texts = open('habr_texts.txt').read().splitlines()
texts = [normalize(tokenize(text.lower())) for text in texts]

CPU times: user 2min 33s, sys: 2.1 s, total: 2min 35s
Wall time: 2min 38s


In [5]:
%%time
texts = open('habr_texts.txt').read().splitlines()
texts = opt_normalize([tokenize(remove_tags(text.lower())) for text in texts], 30000)

CPU times: user 9.01 s, sys: 749 ms, total: 9.76 s
Wall time: 9.94 s


### Тематическое моделирование в gensim

In [14]:
def lda(texts,
        no_above=0.3,
        ngrams=True,
        tfidf=False,
        passes=5,
        eta='auto', 
        iterations=10, 
        num_topics=100):
    if ngrams:
        ph = gensim.models.Phrases(texts, scoring='npmi', threshold=0.4) # threshold можно подбирать
        p = gensim.models.phrases.Phraser(ph)
        texts = p[texts]
    dictionary = gensim.corpora.Dictionary(texts)
    dictionary.filter_extremes(no_above=no_above)
    dictionary.compactify()
    
    corpus = [dictionary.doc2bow(text) for text in texts]
    if tfidf:
        tfidf = gensim.models.TfidfModel(corpus, id2word=dictionary)
        corpus = tfidf[corpus]
    lda_model = gensim.models.LdaModel(corpus=corpus, 
                                       id2word=dictionary,
                                       num_topics=num_topics,
                                       random_state=55,
                                       passes=passes,
                                       eta=eta, 
                                       iterations=iterations)
    return lda_model, texts, corpus

In [15]:
%%time
lda_model, _texts, corpus = lda(texts)

CPU times: user 1min 33s, sys: 2.2 s, total: 1min 35s
Wall time: 1min 35s


Посмотрим на топики.

In [16]:
lda_model.print_topics()

[(89,
  '0.027*"repeat" + 0.023*"paypal" + 0.015*"magento" + 0.012*"узор" + 0.011*"puppet" + 0.011*"bitcoin" + 0.011*"html_pdf" + 0.010*"basic" + 0.009*"consul" + 0.008*"публичный_ключ"'),
 (52,
  '0.028*"уязвимость" + 0.027*"безопасность" + 0.020*"отчёт" + 0.018*"java" + 0.015*"защита" + 0.015*"больший" + 0.013*"проверка" + 0.013*"android" + 0.012*"мобильный_приложение" + 0.011*"анализ"'),
 (31,
  '0.022*"0" + 0.012*"id" + 0.010*"end" + 0.007*"set" + 0.007*"режим" + 0.007*"new" + 0.007*"0_0" + 0.006*"file" + 0.006*"user" + 0.006*"show"'),
 (86,
  '0.051*"символ" + 0.031*"кнопка" + 0.023*"8" + 0.015*"конвертер" + 0.015*"4" + 0.015*"значение" + 0.014*"unsigned_char" + 0.014*"32" + 0.013*"счётчик" + 0.012*"16"'),
 (96,
  '0.031*"событие" + 0.028*"правило" + 0.011*"клиент" + 0.011*"мониторинг" + 0.008*"инцидент" + 0.008*"инфраструктура" + 0.007*"дата-центр" + 0.007*"собственный" + 0.007*"профиль" + 0.007*"лист"'),
 (99,
  '0.014*"yield" + 0.013*"central" + 0.012*"allowed" + 0.012*"этаж_зд

Можно посмотреть метрики.

In [17]:
import numpy as np

In [18]:
lda_model.log_perplexity(corpus[:2000], total_docs=100)

-16.476371184318634

### 3) сделайте хороший словарь (отфильтруйте слишком частотные и редкие слова, попробуйте удалить стоп-слова)

Поработаем над словарём как в Topic_model_BigARTM:

    1) частотные слова - в начале мы убрали русские стоп-слова, ещё хотелось бы убрать частотные английские слова - тоже встречаются
    2) ссылки: "lt;/button&gt", "lt;/script&gt"
    3) отдельные буквы "f"
    4) некоторые слова плохо отсоединены от кавычек (в начале это можно исправить, только не здесь, так как это повлияет на tf-idf): "«зелёный" - исправила при токенизации
    5) знаки препинания стоит убрать (убираются, когда длина слова = 1)

In [19]:
for text in texts:
    for w in text:
        if len(w) == 1 or 'lt;' in w:
            text.remove(w)

In [20]:
%%time
lda_model, _texts, corpus = lda(texts)

CPU times: user 1min 26s, sys: 1.67 s, total: 1min 28s
Wall time: 1min 27s


In [21]:
lda_model.print_topics()

[(90,
  '0.046*"microsoft" + 0.034*"браузер" + 0.025*"респондент" + 0.016*"умолчание" + 0.015*"операционный_система" + 0.015*"edge" + 0.011*"технология" + 0.011*"выбор" + 0.010*"её" + 0.010*"windows_10"'),
 (20,
  '0.029*"оптимизация" + 0.027*"балансировщик_нагрузка" + 0.018*"сервер" + 0.017*"скорость" + 0.016*"обратный" + 0.013*"серверный" + 0.013*"linux" + 0.013*"производительность" + 0.012*"шлюз" + 0.010*"3cx"'),
 (89,
  '0.034*"продукт" + 0.017*"деньга" + 0.016*"рынок" + 0.013*"сайт" + 0.008*"мир" + 0.007*"покупка" + 0.007*"категория" + 0.007*"немного" + 0.007*"платформа" + 0.006*"подборка"'),
 (92,
  '0.017*"устройство" + 0.013*"камера" + 0.012*"модель" + 0.009*"экран" + 0.008*"цена" + 0.007*"телефон" + 0.007*"производитель" + 0.007*"смартфон" + 0.006*"видео" + 0.005*"ноутбук"'),
 (29,
  '0.028*"вакансия" + 0.016*"тестирование" + 0.011*"опыт" + 0.009*"тестировщик" + 0.007*"стадия" + 0.006*"цель" + 0.006*"фич" + 0.006*"выборка" + 0.006*"активный" + 0.006*"вещий"'),
 (80,
  '0.026*"

In [22]:
lda_model.log_perplexity(corpus[:2000], total_docs=100)

-16.876423269963094

### 4) постройте несколько LDA моделей (переберите количество тем, можете поменять eta, alpha, passes), если получаются плохие темы, поработайте дополнительно над предобработкой и словарем;

In [23]:
passes = [5, 10, 50]
topics = [10, 30, 50, 100, 200]

In [ ]:
for t in topics:
    for p in passes:
        lda_model, _texts, corpus = lda(texts, passes=p, num_topics=t)
        print('TOPICS:%s\nPASSES:%s\nSCORE:%s\n%s' % (t, p, lda_model.log_perplexity(corpus[:2000], total_docs=100), lda_model.print_topics(10)))

TOPICS:10
PASSES:5
SCORE:-11.221024036121824
[(0, '0.012*"услуга" + 0.007*"сеть" + 0.006*"доступ" + 0.005*"заказчик" + 0.005*"технология" + 0.005*"организация" + 0.005*"сотрудник" + 0.005*"бизнес" + 0.004*"клиент" + 0.004*"страна"'), (1, '0.026*"игра" + 0.012*"игрок" + 0.005*"играть" + 0.004*"исследование" + 0.004*"понять" + 0.004*"опыт" + 0.004*"день" + 0.004*"вакансия" + 0.003*"доклад" + 0.003*"какой-то"'), (2, '0.011*"устройство" + 0.005*"машина" + 0.004*"сигнал" + 0.004*"температура" + 0.003*"звук" + 0.003*"прибор" + 0.003*"технология" + 0.003*"камера" + 0.003*"датчик" + 0.003*"производство"'), (3, '0.008*"сайт" + 0.008*"клиент" + 0.005*"сеть" + 0.005*"бот" + 0.004*"google" + 0.004*"продажа" + 0.004*"канал" + 0.004*"трафик" + 0.004*"миллион" + 0.004*"реклама"'), (4, '0.008*"инструмент" + 0.006*"продукт" + 0.006*"сайт" + 0.005*"тестирование" + 0.004*"ссылка" + 0.003*"какой-то" + 0.003*"план" + 0.003*"поиск" + 0.003*"отдельный" + 0.003*"заниматься"'), (5, '0.009*"изображение" + 0.006

TOPICS:30
PASSES:50
SCORE:-12.651042998044659
[(17, '0.011*"значение" + 0.010*"точка" + 0.010*"боль" + 0.009*"алгоритм" + 0.008*"сигнал" + 0.008*"изменение" + 0.005*"блок" + 0.005*"объект" + 0.005*"определение" + 0.004*"параметр"'), (9, '0.020*"слово" + 0.018*"студент" + 0.017*"курс" + 0.012*"перевод" + 0.010*"язык" + 0.010*"текст" + 0.007*"обучение" + 0.006*"лекция" + 0.005*"строка" + 0.005*"программирование"'), (19, '0.024*"программа" + 0.018*"microsoft" + 0.016*"платформа" + 0.016*"поддержка" + 0.013*"язык" + 0.012*"windows" + 0.011*"инструмент" + 0.010*"технология" + 0.009*"linux" + 0.008*"java"'), (11, '0.030*"сервис" + 0.019*"кампания" + 0.011*"объявление" + 0.011*"заголовок" + 0.010*"группа" + 0.008*"правило" + 0.006*"хостинг" + 0.006*"adwords" + 0.005*"продукт" + 0.005*"перенос"'), (21, '0.019*"значение" + 0.015*"lt" + 0.014*"цикл" + 0.014*"int" + 0.014*"строка" + 0.013*"переменный" + 0.009*"1" + 0.008*"таблица" + 0.008*"вызов" + 0.007*"0"'), (6, '0.023*"память" + 0.014*"ядро" 

TOPICS:100
PASSES:10
SCORE:-15.957238751400977
[(85, '0.018*"датчик" + 0.011*"выбрать" + 0.010*"порог" + 0.010*"python" + 0.009*"перемещение" + 0.009*"основный" + 0.008*"алгоритм" + 0.008*"линейный" + 0.007*"рекомендация" + 0.007*"оказаться"'), (66, '0.022*"игра" + 0.010*"её" + 0.010*"думать" + 0.009*"vr" + 0.009*"виртуальный_реальность" + 0.009*"дизайнер" + 0.008*"офис" + 0.007*"что-то" + 0.007*"история" + 0.007*"кажется"'), (83, '0.043*"блокчейн" + 0.021*"service_desk" + 0.019*"декабрь_2016" + 0.019*"субъект" + 0.014*"финансовый" + 0.014*"государство" + 0.014*"валюта" + 0.013*"криптовалюта" + 0.013*"ндс" + 0.012*"торговля"'), (28, '0.031*"дом" + 0.028*"устройство" + 0.020*"датчик" + 0.017*"смартфон" + 0.015*"комната" + 0.015*"площадь" + 0.014*"воздух" + 0.012*"xiaomi" + 0.011*"wi-fi" + 0.010*"вода"'), (35, '0.170*"файл" + 0.015*"папка" + 0.012*"временной" + 0.011*"поток" + 0.010*"скрипт" + 0.009*"сайт" + 0.009*"строка" + 0.008*"архив" + 0.008*"путь" + 0.008*"плагин"'), (17, '0.019*"с

### 5) для самой хорошей модели в отдельной ячейке напечатайте 3 хороших (на ваш вкус) темы

### 6) между словарем и обучением модели добавьте tfidf (gensim.models.TfidfModel(corpus, id2word=dictionary); corpus = tfidf[corpus]);
### 7) повторите пункт 4 на преобразованном корпусе;

In [ ]:
for t in topics:
    for p in passes:
        lda_model, _texts, corpus = lda(texts, passes=p, num_topics=t, tfidf=True)
        print('TOPICS:%s\nPASSES:%s\nSCORE:%s\n%s' % (t, p, lda_model.log_perplexity(corpus[:2000], total_docs=100), lda_model.print_topics(10)))

### 8) в отдельной ячейке опишите как изменилась модель (приведите несколько тем, которые стали лучше или хуже, или которых раньше вообще не было; можно привести значения перплексии и когерентности для обеих моделей)

### Разложение матриц в sklearn

In [ ]:
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pandas as pd

Sklearn принимает на вход строки, поэтому склеим наши списки.

In [ ]:
stexts = [' '.join(text) for text in texts]

In [ ]:
def nmf_model(stexts,
              n_components=10,
              alpha=0,
              max_features=25000,
              min_df=5,
              max_df=0.3,
              lowercase=False, 
              ngram_range=(1,1)):
        
    vectorizer = TfidfVectorizer(max_features=max_features,
                                 min_df=min_df,
                                 max_df=max_df,
                                 lowercase=lowercase,
                                 ngram_range=ngram_range)
    X = vectorizer.fit_transform(stexts)
    
    model = NMF(n_components=n_components, random_state=55, alpha=alpha)
    model.fit(X)
    
    return model, vectorizer

In [ ]:
def get_nmf_topics(model, vectorizer, n_top_words):
    
    #id слов.
    feat_names = vectorizer.get_feature_names()
    
    word_dict = {};
    for i in range(model.n_components):
        
        #топ n слов для темы.
        words_ids = model.components_[i].argsort()[:-n_top_words - 1:-1]
        words = [feat_names[key] for key in words_ids]
        word_dict['Topic # ' + '{:02d}'.format(i+1)] = words;
    
    return pd.DataFrame(word_dict);

In [ ]:
%%time
model, vectorizer = nmf_model(stexts)

In [ ]:
get_nmf_topics(model, vectorizer, 10)

### 9) проделайте такие же действия для NMF (образец в конце тетрадки), для построения словаря воспользуйтесь возможностями Count или Tfidf Vectorizer (попробуйте другие значение max_features, min_df, max_df, сделайте нграмы через ngram_range, если хватает памяти), попробуйте такие же количества тем

In [ ]:
for t in topics:
    model, vectorizer = nmf_model(stexts,
                                  n_components=t,
                                  ngram_range=(1,2))
    print('TOPICS:%s\n%s' % (t, get_nmf_topics(model, vectorizer, 10)))

### 10) в отдельной ячейки напечатайте таблицу с темами лучшей NMF модели, сравните их с теми, что получились в LDA.